In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F
import torch
import time
model_name = 'deberain/gpt-neo-125M-fine-tuned-on-ChatGPT-tweets'
#model_name = '/Users/1112082/Desktop/openAI/llama2/llamas/gpt-neo_train/pretrained'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

EOS_TOKEN_ID = 50256

/Users/1112082/Desktop/openAI/env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [20]:
def llm1(prompt, max_new_tokens, do_sample, temperature, top_k, top_p, repetition_penalty):
    start_time = time.time()
    model_inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        input_ids=model_inputs['input_ids'], 
        pad_token_id=EOS_TOKEN_ID,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    gen_txt = tokenizer.decode(output[0], skip_special_tokens=True)
    print(gen_txt,'\nelapsed time:',time.time()-start_time)
    

In [3]:
def decoder2(token_ids):
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    return ''.join(tokens).replace('Ġ',' ').replace('Ċ','\n')

In [21]:
def llm2(
    prompt, 
    max_new_tokens=20, 
    do_sample=True,
    temperature=1,
    top_k = 0,
    top_p = 1,
    repetition_penalty = 1
):
    start_time = time.time()
    model_inputs = tokenizer(prompt, return_tensors="pt")
    gen_token_ids = model_inputs['input_ids']

    output_attentions = ()
    output_hidden_states = ()
    
    with torch.no_grad():
        #iterate up to 'max_new_tokens'
        for i in range(max_new_tokens):
            inp = model.forward(
                gen_token_ids
            )
            
            #repetition penalty process
            logits = inp.logits[0][-1]
            score  = torch.gather(logits, 0, gen_token_ids[0])
            score  = torch.where(score < 0, score * repetition_penalty, score / repetition_penalty)
            logits = logits.scatter_(0,gen_token_ids[0],score)
            
            #if 'do_sample' is True
            if do_sample:
                logits = logits / temperature #'temperature' works only in case 'do_sample' is True            
                if top_k > 0:
                    indices_to_remove = logits < torch.min( torch.topk(logits, top_k)[0] )
                    logits = logits.masked_fill(indices_to_remove, -float("Inf"))            
                if 0 <= top_p <= 1.0:
                    sorted_logits, sorted_indices = torch.sort(logits, descending=False)
                    cumulative_probs = sorted_logits.softmax(dim=0).cumsum(dim=0)
                    sorted_indices_to_remove = cumulative_probs <= (1 - top_p)
                    indices_to_remove = sorted_indices_to_remove.scatter(0, sorted_indices, sorted_indices_to_remove) # dim, index, value                
                    logits = logits.masked_fill(indices_to_remove, -float("Inf"))
                probs = logits.softmax(dim=0) 
                token_id = torch.multinomial(probs, num_samples=1).squeeze(0)
            else:
                logits = logits.softmax(dim=0)
                token_id = torch.argmax(logits, dim=0)
            
            #EOS Token ID 가 예측되면 max_new_tokens 에 도달하지 못했어도 종료
            if token_id == EOS_TOKEN_ID:
                break
            gen_token_ids = torch.cat( (gen_token_ids, torch.tensor([[token_id]]) ), dim=1)
        
    gen_txt = decoder2(gen_token_ids[0])
    print(gen_txt,'\nelapsed time:',time.time()-start_time)
    

In [22]:
prompt = "What is gravity"
prompt_template = "Question: {prompt}\nAnswer:"

In [25]:
#llm1 : 원본 , llm2 & decoder2 : Custom. 
llm1(prompt_template.format(prompt=prompt), max_new_tokens=50,do_sample=True,temperature=1, top_k = 50, top_p=0.9, repetition_penalty=10.0)
print('=============')
llm2(prompt_template.format(prompt=prompt), max_new_tokens=50,do_sample=True,temperature=1, top_k = 50, top_p=0.9, repetition_penalty=10.0)

Q: What is gravity
A: Gravity in air has an average of 1.5, while in water it can range from 0 to 3/2! 
elapsed time: 0.6250591278076172
Q: What is gravity
A: Gravity occurs when an object is thrown at its gravitational acceleration. 
elapsed time: 0.7758758068084717
